# Befolkningsutveckling på DeSO, Stockholms län

## Inledning
Detta är ett skript för att hämta befolkningsstatistik med API från SCB för demografiska statistikområden (DeSO) inom Stockholms län.

## Hämta data från SCB
För att hämta statistik från SCB med API används biblioteket Pyscbwrapper.

Läs mer om Pyscbwrapper här: https://github.com/kirajcg/pyscbwrapper/blob/master/pyscbwrapper.ipynb

In [1]:
#Importera bibliotek
from pyscbwrapper import SCB
import pandas as pd

In [2]:
#Initiera SCB
scb = SCB('sv')

#Välj tabell
scb.go_down('BE', 'BE0101', 'BE0101Y', 'FolkmDesoKonN')

In [3]:
#Ladda hem data

import re
#Använd re för att filtrera vilken/vilka region(er) du vill hämta data för
regioner = scb.get_variables()['region']
r = re.compile('^01')
deso = list(filter(r.match, regioner))

# Skapa en fråga att skicka till databasen
scb.set_query(region = deso,
              kön = ['totalt'])

# Inspektera fråga
scb.get_query()

# Hämta metadata och extrahera data
scb_metadata = scb.get_data()
scb_data = scb_metadata['data']

# Inspektera endast de främsta raderna av data i JSON format
num_rows = 3

# Inspektera de tre första raderna
for row in scb_data[:num_rows]:
    print(row)

{'key': ['01', '1+2', '2010'], 'values': ['2054343']}
{'key': ['01', '1+2', '2011'], 'values': ['2091473']}
{'key': ['01', '1+2', '2012'], 'values': ['2127006']}


In [4]:
# Konvertera en lista av dictionaries till en dictionary med dictionaries
scb_dict = {tuple(d['key']): d['values'][0] for d in scb_data}

# Skapa en DataFrame av dictionary
df = pd.DataFrame.from_dict(scb_dict, orient='index', columns=['value'])

# Sätt ett namn för index
df.index.set_names('keys', inplace=True)
df.reset_index(inplace=True)
df[['Deso', 'kön', 'år']] = df['keys'].apply(pd.Series)
df.drop('keys', axis=1, inplace=True)
df.rename(columns={'value':'befolkning'}, inplace=True)

#Extrahera endast DeSO områden (ett DeSO områdes kod har 9 karaktärer)
mask = (df['Deso'].str.len() == 9)
df['Deso'] = df['Deso'].loc[mask]
df = df.dropna(subset='Deso', how='any')
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)

df.head()

,befolkning,Deso,kön,år
0,728,0114A0010,1+2,2010
1,753,0114A0010,1+2,2011
2,762,0114A0010,1+2,2012
3,789,0114A0010,1+2,2013
4,806,0114A0010,1+2,2014


In [5]:
#Lägg till kommunnamn som kolumn

# Definiera mappning
kommun_mapping = {
    '0114': 'Upplands Väsby',
    '0115': 'Vallentuna',
    '0117': 'Österåker',
    '0120': 'Värmdö',
    '0123': 'Järfälla',
    '0125': 'Ekerö',
    '0126': 'Huddinge',
    '0127': 'Botkyrka',
    '0128': 'Salem',
    '0136': 'Haninge',
    '0138': 'Tyresö',
    '0139': 'Upplands-Bro',
    '0140': 'Nykvarn',
    '0160': 'Täby',
    '0162': 'Danderyd',
    '0163': 'Sollentuna',
    '0180': 'Stockholm',
    '0181': 'Södertälje',
    '0182': 'Nacka',
    '0183': 'Sundbyberg',
    '0184': 'Solna',
    '0186': 'Lidingö',
    '0187': 'Vaxholm',
    '0188': 'Norrtälje',
    '0191': 'Sigtuna',
    '0192': 'Nynäshamn'
}

# Skapa 'kommun' variabel baserad på 'deso'
df['kommun'] = df['Deso'].str[:4].map(kommun_mapping)
df.drop('kön', axis=1, inplace=True)

df.head()

,befolkning,Deso,år,kommun
0,728,0114A0010,2010,Upplands Väsby
1,753,0114A0010,2011,Upplands Väsby
2,762,0114A0010,2012,Upplands Väsby
3,789,0114A0010,2013,Upplands Väsby
4,806,0114A0010,2014,Upplands Väsby
